## SVM + RBF Kernel (no Hyper-Parameter Tuning)

Import the packages.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score

/home/bsknight/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Read in the data and extract features from labels. 

In [2]:
df = pd.read_csv('./Data/munged_df.csv', index_col='account_id')
feature_cols = list(df.columns[:-1])
target_col = df.columns[-1] 
X_all = df[feature_cols]
y_all = df[target_col] 

Scale the features.

In [3]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_all = scaler.fit_transform(X_all)

Split the data into training and testing.

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all , 
                                                    y_all, 
                                                    stratify=y_all,
                                                    test_size=0.1, 
                                                    random_state=1)
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 14946 samples.
Testing set has 1661 samples.


Create the classifier.

In [8]:
from sklearn.svm import LinearSVC
clf = svm.LinearSVC()

In [9]:
from sklearn.metrics import precision_recall_curve
model = clf.fit(X_train, y_train)
default_y_score = model.decision_function(X_test)
default_precision, default_recall, default_thresholds = precision_recall_curve(y_test, default_y_score)

In [10]:
from sklearn.metrics import average_precision_score
default_average_precision = average_precision_score(y_test, default_y_score)

In [28]:
# lw = 2
# plt.figure(figsize=(5,5))
# plt.plot(recall, precision, lw=lw, color='navy', label='Precision-Recall Curve')
# plt.legend(loc="upper right")
# plt.xlabel('Recall: Share of Customer Conversions Captured')
# plt.ylabel('Precision: Accuracy of Predicted Customer Conversions')
# plt.title('Linear SVM with Default Settings', y=1.05, fontsize=16)
# plt.suptitle('Precision-Recall AUC={0:0.2f}'.format(default_average_precision), y=0.92, fontsize=12)
# plt.show()
# # path = '/home/bsknight/Documents/Personal_Training_Git/Udacity/Udacity-Machine_Learning_Nanodegree/' + \
# #        'Capstone_Project/Images'
# # savepath = os.path.join(path, 'non-optimized_linear_svm.png')
# # plt.savefig(savepath)
# # plt.clf()

Derive the F2 score.

In [29]:
from sklearn.metrics import fbeta_score
y_pred = model.predict(X_test)
fbeta_score(y_test, y_pred, average='binary', beta=2)

0.082742316784869985

In [11]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)
from sklearn.cross_validation import cross_val_score
cv_results = pd.DataFrame(cross_val_score(model, X_test, y_test, cv=100, scoring=ftwo_scorer))
cv_results.mean()

/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metric

0    0.173571
dtype: float64

## Hyper-Parameter Tuning

Create the custom scorer.

In [31]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [32]:
def svccv(C):
    return cross_val_score(svm.LinearSVC(C=C, random_state=1),
                           X_train, y_train, ftwo_scorer, cv=20, n_jobs=-1).mean()

if __name__ == "__main__":
    gp_params = {"alpha": 1e5}

    svcBO = BayesianOptimization(svccv, {'C': (0.0001, 1000)})
    svcBO.explore({'C': [0.00001, 0.01, 0.1]})

    svcBO.maximize(n_iter=10, **gp_params)
    print('SVC: %11.9f' % svcBO.res['max']['max_val'])

Initialization
-----------------------------------------
 Step |   Time |      Value |         C | 
    1 | 00m08s |    0.03351 |    0.0000 | 
    2 | 00m08s |    0.04953 |    0.0100 | 
    3 | 00m08s |    0.08954 |    0.1000 | 
    4 | 00m08s |    0.26134 |  864.0092 | 
    5 | 00m08s |    0.23743 |  484.1187 | 
    6 | 00m08s |    0.24957 |  860.8844 | 
    7 | 00m08s |    0.22271 |  790.7968 | 
    8 | 00m08s |    0.24412 |  927.3550 | 
Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         C | 
    9 | 00m09s |    0.22687 |  999.9974 | 
   10 | 00m08s |    0.04558 |    0.0066 | 
   11 | 00m08s |    0.22687 |  999.9761 | 
   12 | 00m09s |    0.03784 |    0.0022 | 
   13 | 00m09s |    0.22687 |  999.9833 | 
   14 | 00m08s |    0.02988 |    0.0006 | 
   15 | 00m08s |    0.22687 |  999.9973 | 
   16 | 00m09s |    0.05477 |    0.0151 | 
   17 | 00m09s |    0.22687 |  999.9987 | 
   18 | 00m08s |    0.05463 |    0.0143 | 
SVC: 0.261341592


In [33]:
results = svcBO.res['max']
svm_C = results['max_params']['C']
print(svm_C)

864.009245032


In [12]:
svm_C = 899.345366365

In [13]:
clf = svm.LinearSVC(C=svm_C)

In [14]:
from sklearn.metrics import precision_recall_curve
optimized_model = clf.fit(X_train, y_train)
optimized_y_score = optimized_model.decision_function(X_test)
optimized_precision, optimized_recall, optimized_thresholds = precision_recall_curve(y_test, optimized_y_score)

In [15]:
from sklearn.metrics import average_precision_score
optimized_average_precision = average_precision_score(y_test, optimized_y_score)

In [41]:
lw = 2
plt.figure(figsize=(6,5))
plt.plot(default_recall, default_precision, 
         lw=lw, color='C3', label='AUC with No Hyper-Parameter Tuning={0:0.2f}'.format(default_average_precision))
plt.plot(optimized_recall, optimized_precision, lw=lw, color='C1', 
         label='AUC with Bayesian Optimization={0:0.2f}'.format(optimized_average_precision))
plt.legend(loc="upper right")
plt.xlabel('Recall: Share of Customer Conversions Captured')
plt.ylabel('Precision: Accuracy of Predicted Customer Conversions')
plt.title('Precision-Recall Curve: Linear SVM', y=1, fontsize=16)
# plt.show()
path = '/home/bsknight/Documents/Personal_Training_Git/Udacity/Udacity-Machine_Learning_Nanodegree/' + \
       'Capstone_Project/Images'
savepath = os.path.join(path, 'Linear_SVM.png')
plt.savefig(savepath)
plt.clf()

Derive the F2 score.

In [39]:
from sklearn.metrics import fbeta_score
y_pred = optimized_model.predict(X_test)
fbeta_score(y_test, y_pred, average='binary', beta=2)

0.17751479289940827

In [16]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)
from sklearn.cross_validation import cross_val_score
cv_results = pd.DataFrame(cross_val_score(optimized_model, X_test, y_test, cv=100, scoring=ftwo_scorer))
cv_results.mean()

/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bsknight/.local/lib/python3.5/site-packages/sklearn/metric

0    0.268472
dtype: float64